## Extraction Algorithm

### Note on how to use the algorithm :
Create a main folder. Inside the main folder create a folder called **form-samples**. Put all the images in form-samples folder. Put this code in the main folder and run it. This creates another folder called **crop** inside the main folder and puts all the extracted feature images of the first file in form-samples to **crop1** folder, all the extracted feature images of the second file in form-samples to **crop2** folder and so on. All the crop*k*(crop1, crop2 etc.) folders are inside the crop folder.

In [2]:
import time
start = time.time()
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import glob
print(time.time()-start,'s')

6.60194730758667 s


In [3]:
def order_points(pts,crop,main=0):
    ''' 
        The function returns the 4 corner points in a set of points and returns the rectangle formed by the four
        co-ordinates. 
        
        The input are a set of points (pts).
        
        'crop' is the amount of pixels that are subtracted from the points obtained which is kind of padding 
        so that we don't lose the data that run outside the area specified.
        
        'main' is the flag which exclusively runs when the ordering points are of the main form. If main is 1, then
        the cropping should be more (This is specific to the form that we have chosen)
    '''
    rect = np.zeros((4, 2), dtype = "float32")
 

    s = pts.sum(axis = 1)

    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
 

    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    if main:
        rect[1][1]-=110
        rect[0][0] -=20
    
    #cropping a little more towards left
    rect[0][0] -=crop
    rect[3][0] -=crop
    #cropping a little more towards right
    rect[1][0] +=crop
    rect[2][0] +=crop
    #cropping a little more towards bottom
    rect[3][1] +=crop
    rect[2][1] +=crop
 
    return rect

In [217]:
def get_outer_points(pts,crop):
    ''' 
        The function returns the 4 corner points in a set of points but in a different way and returns the rectrangle 
        formed from the four co-ordinates. 
        
        The input are a set of points (pts).
        
        'crop' is the amount of pixels that are subtracted from the four points obtained which is kind of padding 
        so that we don't lose the data that run outside the area specified.
        The top left is minimum_x coordinate and minimum y-cordinate.
        The top right is maximum_x coordinate and minimum y-cordinate and so on.
    '''

    rect = np.zeros((4, 2), dtype = "float32")
 
    min_x = min(pts[:,0])
    min_y = min(pts[:,1])
    max_x = max(pts[:,0])
    max_y = max(pts[:,1])

    rect[0][0] = min_x
    rect[0][1] = min_y
    rect[2][0] = max_x
    rect[2][1] = max_y
 

    rect[1][0] = max_x
    rect[1][1] = min_y
    rect[3][0] = min_x
    rect[3][1] = max_y
    
    #cropping a little more towards left
    rect[0][0] -=crop
    rect[3][0] -=crop
    #cropping a little more towards right
    rect[1][0] +=crop
    rect[2][0] +=crop
    #cropping a little more towards bottom
    rect[3][1] +=crop
    rect[2][1] +=crop
 
    return rect

In [218]:
def four_point_warp(image, pts,crop,outer=1,main=0):
    ''' 
        The function finds the corner four points of the image and warps the image accordingly.
        
        image : The grayscale image of the scanned form.
        
        pts   : The set of points from which corner points are to be selected to warp.
        
        crop  : The buffer amount of pixels that should be cropped less so that we don't lose the data.
        
        outer : If outer is 1 get_outer_points() function is called, else order_points() function is called.
        
        main  : This is the variable for the function get_outer_points().
    '''
    if outer:
        rect = get_outer_points(pts,crop)
    else :
        rect = order_points(pts,crop,main)
    (tl, tr, br, bl) = rect
    
    # Finding width and height of the image based on corner points
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    # The destination where the image needs to be pasted. 
    # This is the size of the image where the warped image will be put.
    dst = np.array([
              [0, 0],
              [maxWidth - 1, 0],
              [maxWidth - 1, maxHeight - 1],
              [0, maxHeight - 1]], dtype = "float32")
    
    # Warping the image
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
    return warped

In [219]:
def compare_features_orb(gray_main, gray_crop, mask_tl=None, mask_br=None):
    '''
        This function is used to compare the features of two images to match them based on the features detected.
        It detects the features and returns all the good points of the features based on the Lowe's ratio test.
        It uses ORB (Oriented FAST and Rotated BRIEF) algorithm to compare features.
        
        gray_main : The gray-scale image that is to be compared for the features.
        
        gray_crop : The gray-scale image of the standard image with features that is used to compare other images.
        
        The next two arguments are to reduce the computation time
        mask_tl   : This is the top left co-ordinates of a mask that covers the approximate central part of the image 
                    so that it doesn't match features there since we need only the corner points.
        
        mask_br   : This is the bottom right co-ordinates of a mask that covers the approximate central part of the image 
                    so that it doesn't match features there since we need only the corner points.
        
    '''
    #Trying feature matching using ORB
    orb = cv2.ORB_create()
    Mask = None

    if mask_tl and mask_br:
        Mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)
        cv2.rectangle(Mask, mask_tl, mask_br, (255), thickness = -1)
    # find the keypoints and descriptors with SIFT
    kp1, des1 = orb.detectAndCompute(gray_main,mask)
    kp2, des2 = orb.detectAndCompute(gray_crop,None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(des1,des2)

    matches = sorted(matches, key = lambda x:x.distance)

    img = cv2.drawMatches(gray_main,kp1,gray_crop,kp2,matches,None, flags=2)

    return np.float32([ kp1[m.queryIdx].pt for m in matches ])

In [220]:
def compare_features(gray_main, gray_crop, mask_tl=None, mask_br=None, Mask_main=None):
    '''
        This function is used to compare the features of two images to match them based on the features detected.
        It detects the features and returns all the good points of the features based on the Lowe's ratio test.
        It uses Scale-Invariant Feature Transform algorithm to compare features.
        
        gray_main : The gray-scale image that is to be compared for the features.
        
        gray_crop : The gray-scale image of the standard image with features that is used to compare other images.
        
        The next two arguments are to reduce the computation time
        mask_tl   : This is the top left co-ordinates of a mask that covers the approximate central part of the image 
                    so that it doesn't match features there since we need only the corner points.
        
        mask_br   : This is the bottom right co-ordinates of a mask that covers the approximate central part of the image 
                    so that it doesn't match features there since we need only the corner points.
        
        Mask_main : Mask when the main form is to be compared
    '''
    #Trying feature matching using SIFT
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()
    
    Mask=Mask_main
    
    if mask_tl and mask_br:
        if Mask_main is not None:
            cv2.rectangle(Mask, mask_tl, mask_br, (0), thickness = -1)
        else :
            Mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)
            cv2.rectangle(Mask, mask_tl, mask_br, (255), thickness = -1)
    
    
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(gray_main,Mask)
    kp2, des2 = sift.detectAndCompute(gray_crop,None)
    
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    matches = flann.knnMatch(des1,des2,k=2,mask=Mask)
    
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.8*n.distance:
            good.append(m)
    
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    
    h,w = gray_main.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    
    
    img2 = cv2.polylines(gray_crop,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

    img3 = cv2.drawMatches(gray_main,kp1,gray_crop,kp2,good,None,**draw_params)
    return np.float32([ kp1[m.queryIdx].pt for m in good ])

In [221]:
def read_input(main_file, crop_file):
    '''
        This function reads the input(main_file is the scanned image and crop_file is the standard image used 
         for comparision) and returns the required output.
    '''
    img1 = cv2.imread(main_file)
    crop = cv2.imread(crop_file)
    gray_main = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    gray_crop = cv2.cvtColor(crop,cv2.COLOR_BGR2GRAY)
    height_main, width_main = gray_main.shape
    height_crop, width_crop = gray_crop.shape
    height = height_main
    width = width_main

    return img1, gray_main, gray_crop, height, width

In [222]:
def crop_selected_rectangle(img, tl, br):
    '''
        Given top left(tl) and bottom right (br) x and y co-ordinates of the image(img) returns the cropped image of the 
        co-ordinates.
    '''
    return img[tl[1]:br[1],tl[0]:br[0]]

In [226]:
'''
    This is the main code snippet that calls all the required functions and extracts the features and stores them.
'''
start_time = time.time()

types = ('*.jpg', '*.JPG','*.JPEG', '*.jpeg')
files = []
os.system("mkdir crop")
os.chdir(os.getcwd() + "/form-samples/")

# Takes all the files in form-samples
for t in types:
    if glob.glob(t) != [] :
        files.append(glob.glob(t))

# Creates crop folders
os.chdir("../crop/")
i=1
for f in files[0]:
    print(f)
    # Reading input
    img1, gray_main, gray_crop, height, width = read_input('../form-samples/'+f, '../form_print.jpg')
    mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)+255
    
    # Get all the feature points after comparison
    points = compare_features(gray_main, gray_crop, (0,int(height/4)),(width,int(18*height/25)),mask)
    
    crop = 30
    # Warp the image based on the points obtained after feature extraction.
    dst = four_point_warp(gray_main, points, crop, outer=0, main=1)
    height, width = dst.shape
    
    os.system("mkdir crop"+str(i))
    # Crop the required fields like branch_name, date, Account number.
    branch = crop_selected_rectangle(dst,(int(width/20),int(height/20)),(int(7*width/11),int(3*height/37)))
    date = crop_selected_rectangle(dst,(int(25*width/37),int(height/21)),(int(width),int(3*height/37)))
    ac_no = crop_selected_rectangle(dst,(0,int(height/12)),(int(23*width/44),int(19*height/160)))
    name = crop_selected_rectangle(dst,(int(width/70),int(37*height/192)),(int(31*width/34),int(11*height/50)))
    print('Saving into crop'+str(i))
    os.chdir(os.getcwd()+'/crop'+str(i))
    
    # Writing everything to files
    cv2.imwrite('branch.jpg',branch)
    cv2.imwrite('Date.jpg',date)
    cv2.imwrite('Ac_no.jpg',ac_no)
    cv2.imwrite('Name.jpg',name)
    os.chdir('../')
    print('Time taken : ',time.time()-start_time)
    i=i+1


filled-pa.jpg
Saving into crop1
Time taken :  87.8336250782013
filled-s1.jpg
Saving into crop2
Time taken :  156.4682605266571
filled-s2.jpg
Saving into crop3
Time taken :  219.932514667511


#### The rest of the cells are for coder reference.

In [212]:
# crop = 30
# dst = four_point_warp(gray_main, points, crop, outer=0, main=1)
# print(time.time()-start_time)
# height, width = dst.shape
# print(height,width)

3742.9271965026855
2902 2062


In [213]:
# Crop branch
# cv2.rectangle(dst,(int(width/20),int(height/20)),(int(7*width/11),int(3*height/37)),(0),3)
# branch = crop_selected_rectangle(dst,(int(width/20),int(height/20)),(int(7*width/11),int(3*height/37)))
# date
# cv2.rectangle(dst,(int(25*width/37),int(height/21)),(int(width),int(3*height/37)),(0),3)
# Ac no.
# cv2.rectangle(dst,(0,int(height/12)),(int(23*width/44),int(19*height/160)),(0),3)
# cv2.rectangle(dst,(int(width/70),int(37*height/192)),(int(31*width/34),int(11*height/50)),(0),3)
# cv2.imwrite('Out.jpg',dst)

True

In [227]:
# i=0
# os.system("mkdir crop")
# os.chdir('..')
# os.system("mkdir crop"+str(i))
# print(os.getcwd())

/home/darshan/CS/Summer2018/Impel/OCR/Darshan/Photos/crop


'a'

In [73]:
# For residential address
# address_x = (int(0),int(18*height/25))
# address_y = (width,int(25*height/26))

# points = compare_features(gray_main, gray_crop, address_x, address_y)
# points = points[points[:,1]>(7*height)/10]